# Tests on Grid generation & extension

What follows are jax samples based on pykan or efficientkan, as far as the grid generation and extension process is concerned, to get a better understanding of what's going on.

In [4]:
import jax.numpy as jnp

In [2]:
# Input parameters relevant to grid
k = 3 # splines order
G = 5 # grid size (number of knot vector elements)
n_in = 2 # number of layer's input nodes
n_out = 3 # number of layer's output nodes

**<span style="color:red">Open Question: Why are we starting with a grid defined only by its range?</span>**

In [3]:
# Grid Initialization - the grid is originally just a knot vector
grid_range = [-1, 1]

**<span style="color:red">Open Question: Why are we augmenting the grid like this instead of appending the first and last entries k times at the start and end?</span>**

In [4]:
# Grid augmentation
h = (grid_range[-1] - grid_range[0]) / (G-1)
grid = (jnp.arange(-k, G + k + 1, dtype=jnp.float32) * h + grid_range[0])

grid

Array([-2.5, -2. , -1.5, -1. , -0.5,  0. ,  0.5,  1. ,  1.5,  2. ,  2.5,
        3. ], dtype=float32)

In [5]:
# Expand for broadcasting - the shape must be (layer_size, G + k), so that the grid
# can be passed in get_spline_basis and return a Bi(x) array, where we need only multiply
# with the coefficients to get the full spline(x), for each layer's spline (n_in * n_out total).
grid = jnp.expand_dims(grid, axis=0)
grid = jnp.tile(grid, (n_in*n_out, 1))

grid

Array([[-2.5, -2. , -1.5, -1. , -0.5,  0. ,  0.5,  1. ,  1.5,  2. ,  2.5,
         3. ],
       [-2.5, -2. , -1.5, -1. , -0.5,  0. ,  0.5,  1. ,  1.5,  2. ,  2.5,
         3. ],
       [-2.5, -2. , -1.5, -1. , -0.5,  0. ,  0.5,  1. ,  1.5,  2. ,  2.5,
         3. ],
       [-2.5, -2. , -1.5, -1. , -0.5,  0. ,  0.5,  1. ,  1.5,  2. ,  2.5,
         3. ],
       [-2.5, -2. , -1.5, -1. , -0.5,  0. ,  0.5,  1. ,  1.5,  2. ,  2.5,
         3. ],
       [-2.5, -2. , -1.5, -1. , -0.5,  0. ,  0.5,  1. ,  1.5,  2. ,  2.5,
         3. ]], dtype=float32)

This is the initialization of the grid per layer (registered at `self.`). Then, we have the `upgrade_grid` routine, which is supposed to perform an update for each layer's grid (**can be trainable or not in the future**).

In pykan, the `update_grid_from_samples` routine simply performs the mix of adaptive and uniform sampling, given the current grid and the batch. It is the `initialize_grid_from_parent` routine that performs the grid extension. On the other hand, in efficientkan both are handled together in the `update_grid` routine. In what follows, we try to follow the approach of efficientkan, however working with grids of shape $(n_{in}\cdot n_{out},\text{batch})$ instead of $(n_{in},\text{batch})$.

**<span style="color:red">IDEA: Keep splines as model param function, same as efficientkan, because the grid is also. Then $k$ can be inherited from self too, and the whole thing can normally be jitted.</span>** 

## Updating the Grid

Steps:

* 

In [2]:
import torch

In [5]:
batch = 300

x = jnp.array([[1,2,4,6,8],[1,3,9,66,33]])

In [7]:
type(x)

jaxlib.xla_extension.ArrayImpl

In [ ]:
def curve2coeff(self, x: torch.Tensor, y: torch.Tensor):
    """
    Compute the coefficients of the curve that interpolates the given points.

    Args:
        x (torch.Tensor): Input tensor of shape (batch_size, in_features).
        y (torch.Tensor): Output tensor of shape (batch_size, in_features, out_features).

    Returns:
        torch.Tensor: Coefficients tensor of shape (out_features, in_features, grid_size + spline_order).
    """
    assert x.dim() == 2 and x.size(1) == self.in_features
    assert y.size() == (x.size(0), self.in_features, self.out_features)

    A = self.b_splines(x).transpose(
        0, 1
    )  # (in_features, batch_size, grid_size + spline_order)
    B = y.transpose(0, 1)  # (in_features, batch_size, out_features)
    solution = torch.linalg.lstsq(
        A, B
    ).solution  # (in_features, grid_size + spline_order, out_features)
    result = solution.permute(
        2, 0, 1
    )  # (out_features, in_features, grid_size + spline_order)

    assert result.size() == (
        self.out_features,
        self.in_features,
        self.grid_size + self.spline_order,
    )
    
    return result

In [ ]:
def curve2coef(x_eval, y_eval, grid, k, device="cpu"):
    '''
    converting B-spline curves to B-spline coefficients using least squares.
    
    Args:
    -----
        x_eval : 2D torch.tensor
            shape (number of splines, number of samples)
        y_eval : 2D torch.tensor
            shape (number of splines, number of samples)
        grid : 2D torch.tensor
            shape (number of splines, number of grid points)
        k : int
            the piecewise polynomial order of splines.
        device : str
            devicde
    '''
    # x_eval: (size, batch); y_eval: (size, batch); grid: (size, grid); k: scalar
    mat = B_batch(x_eval, grid, k, device=device).permute(0, 2, 1)
    # coef = torch.linalg.lstsq(mat, y_eval.unsqueeze(dim=2)).solution[:, :, 0]
    coef = torch.linalg.lstsq(mat.to(device), y_eval.unsqueeze(dim=2).to(device),
                              driver='gelsy' if device == 'cpu' else 'gels').solution[:, :, 0]
    return coef.to(device)

def curve2coeff(x: torch.Tensor, y: torch.Tensor):
    """
    Compute the coefficients of the curve that interpolates the given points.

    Args:
        x (torch.Tensor): Input tensor of shape (batch_size, in_features).
        y (torch.Tensor): Output tensor of shape (batch_size, in_features, out_features).

    Returns:
        torch.Tensor: Coefficients tensor of shape (out_features, in_features, grid_size + spline_order).
    """
    assert x.dim() == 2 and x.size(1) == self.in_features
    assert y.size() == (x.size(0), self.in_features, self.out_features)

    A = self.b_splines(x).transpose(
        0, 1
    )  # (in_features, batch_size, grid_size + spline_order)
    B = y.transpose(0, 1)  # (in_features, batch_size, out_features)
    solution = torch.linalg.lstsq(
        A, B
    ).solution  # (in_features, grid_size + spline_order, out_features)
    result = solution.permute(
        2, 0, 1
    )  # (out_features, in_features, grid_size + spline_order)

    assert result.size() == (
        self.out_features,
        self.in_features,
        self.grid_size + self.spline_order,
    )
    return result.contiguous()

In [19]:
c_basis = self.param('c_basis', lambda rng, shape: self.noise_std * jax.random.normal(rng, shape), (self.n_in * self.n_out, self.G + self.k))

In [22]:
noises

tensor([[ 0.0058,  0.0021,  0.0076,  0.0028, -0.0005,  0.0089,  0.0088,  0.0095,
          0.0024,  0.0081, -0.0098, -0.0083],
        [-0.0080,  0.0051, -0.0024, -0.0011,  0.0025,  0.0066, -0.0058,  0.0025,
         -0.0008,  0.0042,  0.0055, -0.0026],
        [ 0.0077,  0.0016,  0.0033, -0.0017,  0.0016,  0.0061,  0.0098,  0.0029,
         -0.0022,  0.0065,  0.0022, -0.0014],
        [-0.0082, -0.0047, -0.0081,  0.0032,  0.0077,  0.0088,  0.0060,  0.0032,
         -0.0013, -0.0062,  0.0018, -0.0087],
        [-0.0060,  0.0015, -0.0003,  0.0037,  0.0091,  0.0044,  0.0088, -0.0012,
         -0.0011,  0.0041,  0.0025, -0.0051],
        [-0.0078,  0.0031,  0.0060, -0.0075,  0.0048,  0.0054, -0.0027,  0.0076,
         -0.0004,  0.0037, -0.0088,  0.0054],
        [-0.0032, -0.0082, -0.0009, -0.0058, -0.0040, -0.0002,  0.0004,  0.0067,
         -0.0006, -0.0057,  0.0079,  0.0091],
        [-0.0100, -0.0012,  0.0075,  0.0058,  0.0074,  0.0092, -0.0081, -0.0080,
         -0.0025, -0.0071,  0.

In [1]:
import jax
import jax.numpy as jnp
import math
from flax import linen as nn
from flax.training import train_state
from flax.linen import initializers

class KANLayer(nn.Module):
    
    n_in: int = 2
    n_out: int = 5
    G: int = 5
    grid_range: tuple = (-1, 1)
    k: int = 3

    const_spl: float or bool = False
    const_res: float or bool = False
    residual: nn.Module = nn.swish
    
    noise_std: float = 0.1
    grid_e: float = 0.02

    
    def setup(self):
        # Calculate the step size for the knot vector based on its end values
        h = (self.grid_range[1] - self.grid_range[0]) / (self.G - 1)

        # Create the initial knot vector and perform augmentation
        # Now it is expanded from G+1 points to G+1 + 2k points, because k points are appended at each of its ends
        grid = jnp.arange(-self.k, self.G + self.k + 1, dtype=jnp.float32) * h + self.grid_range[0]
        
        # Expand for broadcasting - the shape becomes (n_in*n_out, G + 2k + 1), so that the grid
        # can be passed in all n_in*n_out spline basis functions simultaneously
        grid = jnp.expand_dims(grid, axis=0)
        grid = jnp.tile(grid, (self.n_in*self.n_out, 1))

        # Store the grid as a non trainable variable
        self.grid = self.variable('state', 'grid', lambda: grid)
        
        # Register & initialize the spline basis functions' coefficients as trainable parameters
        # They are drawn from a normal distribution with zero mean and an std of noise_std
        self.c_basis = self.param('c_basis', initializers.normal(stddev=self.noise_std), (self.n_in * self.n_out, self.G + self.k))
        
        # If const_spl is set as a float value, treat it as non trainable and pass it to the c_spl array with shape (n_in*n_out)
        # Otherwise register it as a trainable parameter of the same size and initialize it
        if isinstance(self.const_spl, float):
            self.c_spl = jnp.ones(self.n_in*self.n_out) * self.const_spl
        elif self.const_spl is False:
            self.c_spl = self.param('c_spl', initializers.constant(1.0), (self.n_in * self.n_out,))

        # If const_res is set as a float value, treat it as non trainable and pass it to the c_res array with shape (n_in*n_out)
        # Otherwise register it as a trainable parameter of the same size and initialize it
        if isinstance(self.const_res, float):
            self.c_res = jnp.ones(self.n_in * self.n_out) * self.const_res
        elif self.const_res is False:
            self.c_res = self.param('c_res', initializers.constant(1.0), (self.n_in * self.n_out,))

    def update_grid(self):
        # Example update function
        self.grid.value = self.grid.value*2.25

    def __call__(self, x):
        # Dummy to see correct grid update
        return x*self.grid.value[0][0]


In [3]:
# Instantiate a layer
kan_layer = KANLayer(n_in=2, n_out=3, G=5, grid_range=(-1,1), k=3)

# Initialization
rng = jax.random.PRNGKey(0)
variables = kan_layer.init(rng, jnp.ones((2, 3)))

# Print the trainable parameters and non-trainable state
print("Trainable parameters:")
print(variables['params'])

print("\nNon-trainable state variables:")
print(variables['state'])

Trainable parameters:
{'c_basis': Array([[ 0.02399894,  0.1846351 , -0.00519568, -0.00863923,  0.0585282 ,
        -0.15037541, -0.01022594, -0.02914488],
       [ 0.04935711,  0.03238467, -0.15360688,  0.12719317,  0.04799316,
         0.05530524, -0.02173939,  0.19702375],
       [-0.06749373,  0.0200477 ,  0.23864464, -0.1581935 ,  0.0072664 ,
        -0.10175776,  0.00515934,  0.01749068],
       [-0.00339971, -0.09349852,  0.07724477,  0.04379462,  0.09954414,
         0.04300999,  0.02973044, -0.01073924],
       [ 0.05613207, -0.06796809,  0.07379916, -0.0517869 , -0.0697948 ,
         0.00100837,  0.05920225, -0.07538771],
       [-0.01342505, -0.02973855, -0.00334156, -0.09072405,  0.01592094,
        -0.04852676,  0.02748392, -0.06365392]], dtype=float32), 'c_spl': Array([1., 1., 1., 1., 1., 1.], dtype=float32), 'c_res': Array([1., 1., 1., 1., 1., 1.], dtype=float32)}

Non-trainable state variables:
{'grid': Array([[-2.5, -2. , -1.5, -1. , -0.5,  0. ,  0.5,  1. ,  1.5,  2. , 

In [4]:
# Simple forward pass
x, _ = kan_layer.apply(variables, jnp.ones((2,3)))

print(x)

[-2.5 -2.5 -2.5]


In [5]:
# No forward pass, but applying the update_grid function
x, new_state = kan_layer.apply(variables, method=kan_layer.update_grid, mutable=['state'])

print(new_state)

{'state': {'grid': Array([[-5.625, -4.5  , -3.375, -2.25 , -1.125,  0.   ,  1.125,  2.25 ,
         3.375,  4.5  ,  5.625,  6.75 ],
       [-5.625, -4.5  , -3.375, -2.25 , -1.125,  0.   ,  1.125,  2.25 ,
         3.375,  4.5  ,  5.625,  6.75 ],
       [-5.625, -4.5  , -3.375, -2.25 , -1.125,  0.   ,  1.125,  2.25 ,
         3.375,  4.5  ,  5.625,  6.75 ],
       [-5.625, -4.5  , -3.375, -2.25 , -1.125,  0.   ,  1.125,  2.25 ,
         3.375,  4.5  ,  5.625,  6.75 ],
       [-5.625, -4.5  , -3.375, -2.25 , -1.125,  0.   ,  1.125,  2.25 ,
         3.375,  4.5  ,  5.625,  6.75 ],
       [-5.625, -4.5  , -3.375, -2.25 , -1.125,  0.   ,  1.125,  2.25 ,
         3.375,  4.5  ,  5.625,  6.75 ]], dtype=float32)}}


In [6]:
# Update variables, pass to the model and retry forward pass with new grid
variables['state'] = new_state['state']

x, state = kan_layer.apply(variables, jnp.ones((2, 3)))

print(x)

[-5.625 -5.625 -5.625]


In [7]:
# Apply the update again
x, new_state = kan_layer.apply(variables, method=kan_layer.update_grid, mutable=['state'])

print(new_state)

{'state': {'grid': Array([[-12.65625, -10.125  ,  -7.59375,  -5.0625 ,  -2.53125,   0.     ,
          2.53125,   5.0625 ,   7.59375,  10.125  ,  12.65625,  15.1875 ],
       [-12.65625, -10.125  ,  -7.59375,  -5.0625 ,  -2.53125,   0.     ,
          2.53125,   5.0625 ,   7.59375,  10.125  ,  12.65625,  15.1875 ],
       [-12.65625, -10.125  ,  -7.59375,  -5.0625 ,  -2.53125,   0.     ,
          2.53125,   5.0625 ,   7.59375,  10.125  ,  12.65625,  15.1875 ],
       [-12.65625, -10.125  ,  -7.59375,  -5.0625 ,  -2.53125,   0.     ,
          2.53125,   5.0625 ,   7.59375,  10.125  ,  12.65625,  15.1875 ],
       [-12.65625, -10.125  ,  -7.59375,  -5.0625 ,  -2.53125,   0.     ,
          2.53125,   5.0625 ,   7.59375,  10.125  ,  12.65625,  15.1875 ],
       [-12.65625, -10.125  ,  -7.59375,  -5.0625 ,  -2.53125,   0.     ,
          2.53125,   5.0625 ,   7.59375,  10.125  ,  12.65625,  15.1875 ]],      dtype=float32)}}


In [8]:
# What happens if we do not pass mutable=['state'] ?
x, new_state = kan_layer.apply(variables, method=kan_layer.update_grid)

ModifyScopeVariableError: Cannot update variable "grid" in "/" because collection "state" is immutable. (https://flax.readthedocs.io/en/latest/api_reference/flax.errors.html#flax.errors.ModifyScopeVariableError)

### A very rough example on how this would look during training

```
import optax  # For the optimizer
from flax.training import train_state

class TrainState(train_state.TrainState):
    grid: jnp.ndarray

def create_train_state(rng, model, learning_rate):
    variables = model.init(rng, jnp.ones((1, 2)), mutable=['non_trainable'])
    params = variables['params']
    grid = variables['non_trainable']['grid']
    tx = optax.adam(learning_rate)
    return TrainState.create(apply_fn=model.apply, params=params, tx=tx, grid=grid)

def train_model(state, model, epochs, data_loader):
    for epoch in range(epochs):
        for batch in data_loader:
            state = train_step(state, batch)
        
        if (epoch + 1) % 5 == 0:
            state = update_grid_state(state, model)
            print(f"Grid updated at epoch {epoch + 1}")

    return state

@jax.jit
def train_step(state, batch):
    def loss_fn(params):
        logits = state.apply_fn({'params': params}, batch['x'])
        loss = jnp.mean((logits - batch['y']) ** 2)
        return loss
    grads = jax.grad(loss_fn)(state.params)
    state = state.apply_gradients(grads=grads)
    return state

@jax.jit
def update_grid_state(state, model):
    variables = {'params': state.params, 'non_trainable': {'grid': state.grid}}
    _, updated_state = model.apply(variables, jnp.ones((1, 2)), method=model.update_grid, mutable=['non_trainable'])
    state = state.replace(grid=updated_state['non_trainable']['grid'])
    return state

# Example data loader
data_loader = [{'x': jnp.ones((1, 2)), 'y': jnp.ones((1, 2))} for _ in range(10)]

# Initialize model and state
rng = jax.random.PRNGKey(0)
kan_layer = KANLayer(n_in=2, n_out=3, G=5, grid_range=(-1, 1), k=3)
state = create_train_state(rng, kan_layer, learning_rate=0.001)

# Train the model
state = train_model(state, kan_layer, epochs=20, data_loader=data_loader)

# Check final grid value
print("Final Grid shape:", state.grid.shape)
print(state.grid)
```